In [ ]:
import json

In [ ]:
with open("../results/EAUC.json", "r") as f:
    EAUC = json.load(f)
    print(EAUC)

In [ ]:
global_gaussian = EAUC["global_gaussian"]
local_gaussian = EAUC["local_gaussian"]
global_mean = EAUC["global_mean"]
local_mean = EAUC["local_mean"]

In [ ]:
local_mean["resnet"]["ECG200"].keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# calculacte area under the curve

models = ["rocket", "resnet", "mrseql"]
datasets = ["SwedishLeaf", "ECG200", "Plane"]
explainers = ["shapley", "kernelshap", "leftist_shap", "leftist_lime", "lime", "mrseql"]
methods = [global_gaussian, global_mean, local_gaussian, local_mean]

EAUCs = {}

explainer = "shapley"

all_eaucs_scaled = []

for method in methods:

    for model in models:
        
        eauc_model = []
        for explainer in explainers:

            eauc = 0
            for dataset in datasets:

                accs = method[model][dataset][explainer]
                eauc += np.trapz(accs, x=np.arange(0.0, 1.01, 0.1))
                
            eauc = round(eauc / len(datasets), 2)
            eauc_model.append(eauc)
        
        eauc_model = np.array(eauc_model)
        eauc_model = np.interp(eauc_model, (eauc_model.min(), eauc_model.max()), (0, 1)).tolist()
        all_eaucs_scaled.append(eauc_model)
    
all_eaucs_scaled = np.array(all_eaucs_scaled)

print(all_eaucs_scaled.shape)
print(all_eaucs_scaled)

In [ ]:
means = np.mean(all_eaucs_scaled, axis=0)
means = np.interp(means, (means.min(), means.max()), (0, 1))
means = 1 - means
means = [str(round(x, 2)) for x in means]
"& " + " & ".join(means)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))

model = "rocket"
dataset = "Plane"

for key, values in global_gaussian[model][dataset].items():

    plt.plot(values, label=key)
plt.legend()

plt.title(f"{model} - {dataset}")

plt.xlabel("k")
plt.ylabel("Accuracy")

plt.show()